In [5]:
# %run /Users/denis/Documents/Projects/scripts/init.ipy
%run /Users/maayanlab/Documents/init.ipy
os.chdir('..')

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [15]:
import json, re

In [10]:
infiles = ['03-articles/bioinformatics_articles.txt', '03-articles/bmc-bioinformatics_articles.txt', '03-articles/database_articles.txt', '03-articles/nar_articles.txt']

In [114]:
# Initialize dataframe
abstract_dataframe = pd.concat([pd.read_table(x)[['abstract', 'doi']] for x in infiles])

# Fix abstract
abstract_dataframe['abstract'] = [' '.join([abstract_tuple[1] for abstract_tuple in json.loads(x)['abstract'] if abstract_tuple[0] not in [u'Contact:', u'Availability and implementation', u'Supplementary information']]) for x in abstract_dataframe['abstract']]

# Get dict
abstract_dict = abstract_dataframe.set_index('doi').to_dict()['abstract']

# Get abstracts
abstracts = abstract_dict.values()

# Get vectorizer
# vect = TfidfVectorizer(min_df=1)

# Compute similarity
# tfidf = vect.fit_transform(abstract_dict.values())

# Get similarity dataframe
# similarity_dataframe = pd.melt(pd.DataFrame((tfidf * tfidf.T).A, columns=abstract_dict.keys(), index=abstract_dict.keys()).reset_index(), id_vars='index').rename(columns={'index': 'source_abstract', 'variable': 'target_abstract'})

In [310]:
# Converts Penn Treebank POS tags to WordNet tags    
def penn_to_wn_tags(pos_tag):
    if pos_tag.startswith('J'):
        return wn.ADJ
    elif pos_tag.startswith('V'):
        return wn.VERB
    elif pos_tag.startswith('N'):
        return wn.NOUN
    elif pos_tag.startswith('R'):
        return wn.ADV
    else:
        return None
    
# Define stopwords
stopwords=nltk.corpus.stopwords.words("english")
stopwords.extend(['www','mail','edu','athttps'])

# Process text
def process_text(text):
    
    # Compile regular expression
    remove_characters = re.compile('[^a-zA-Z ]+')
    
    # Remove special characters and make lowercase
    processed_text=re.sub(remove_characters, r'', text.encode('ascii', 'ignore').strip().lower())
    
    # Tokenize
    processed_text = [x.strip() for x in nltk.word_tokenize(processed_text)]

    # Remove stopwords
    processed_text = [x for x in processed_text if x not in stopwords]
    
    # Stem
    processed_text = [nltk.PorterStemmer().stem(x) for x in processed_text]

    # Tag and make lowercase
#     processed_text = [(word.lower(), penn_to_wn_tags(pos_tag)) for word, pos_tag in tag(" ".join(processed_text))]        
    
    # Check if exists
    processed_text = [x for x in processed_text if nltk.corpus.wordnet.synsets(x)]
    
    # Join
    processed_text = " ".join(processed_text)
    
    # Return
    return processed_text

In [313]:
abstract_dataframe.head()

,abstract,doi
0,The analysis of large biological datasets ofte...,https://doi.org/10.1093/bioinformatics/btu595
1,Chromatin immunoprecipitation (ChIP) followed ...,https://doi.org/10.1093/bioinformatics/btu604
2,Although peak finding in next-generation seque...,https://doi.org/10.1093/bioinformatics/btu568
3,"Large-scale cancer genomic studies, such as Th...",https://doi.org/10.1093/bioinformatics/btu603
4,": In virology, massively parallel sequencing (...",https://doi.org/10.1093/bioinformatics/btu587


In [311]:
# Initialize dataframe
abstract_dataframe = pd.concat([pd.read_table(x)[['abstract', 'doi']] for x in infiles])

# Fix abstract
abstract_dataframe['abstract'] = [' '.join([abstract_tuple[1] for abstract_tuple in json.loads(x)['abstract'] if abstract_tuple[0] not in [u'Contact:', u'Availability and implementation', u'Supplementary information']]) for x in abstract_dataframe['abstract']]

# Process abstracts
processed_abstracts = [process_text(x) for x in abstract_dataframe['abstract']]

# Get vectorized
tfidf_vectorizer = TfidfVectorizer(min_df=0.00,max_df=1.0, ngram_range=(1, 1))

# Get matrix
tfidf_matrix = tfidf_vectorizer.fit_transform(processed_abstracts).astype(float)

#Calculate the adjacency matrix
adjacency_dataframe = pd.DataFrame(sklearn.metrics.pairwise.cosine_similarity(tfidf_matrix, tfidf_matrix), index=abstract_dataframe['doi'], columns=abstract_dataframe['doi'])

# Feature dataframe
feature_dataframe = pd.DataFrame(tfidf_matrix.todense(), index=abstract_dataframe['doi'], columns=tfidf_vectorizer.get_feature_names())

# Melt
feature_dataframe_melted = pd.melt(feature_dataframe.reset_index().rename(columns={'level_0': 'doi'}), id_vars='doi', var_name='word', value_name='importance')

# Get top keywords dataframe
top_keyword_dataframe = feature_dataframe_melted.groupby('doi')['word','importance'].apply(lambda x: x.nlargest(5, columns=['importance'])).reset_index().set_index('doi').drop_duplicates().drop(['level_1', 'importance'], axis=1)
top_keyword_dataframe.head()

,word
doi,
https://doi.org/10.1093/bioinformatics/btu547,browser
https://doi.org/10.1093/bioinformatics/btu547,http
https://doi.org/10.1093/bioinformatics/btu547,plain
https://doi.org/10.1093/bioinformatics/btu547,web
https://doi.org/10.1093/bioinformatics/btu547,bam


In [314]:
adjacency_dataframe.head()

,https://doi.org/10.1093/bioinformatics/btu599,https://doi.org/10.1093/database/baw006,https://doi.org/10.1093/bioinformatics/btu590,https://doi.org/10.1093/bioinformatics/btu593,https://doi.org/10.1093/bioinformatics/btu595,https://doi.org/10.1093/bioinformatics/btu594,https://doi.org/10.1093/bioinformatics/btu596,https://doi.org/10.1093/bioinformatics/btw557,https://doi.org/10.1093/bioinformatics/btw554,https://doi.org/10.1093/nar/gkw925,...,https://doi.org/10.1093/bioinformatics/btw563,https://doi.org/10.1093/nar/gkv1308,https://doi.org/10.1093/nar/gkv1309,https://doi.org/10.1093/bioinformatics/btw564,https://doi.org/10.1093/bioinformatics/btw565,https://doi.org/10.1093/bioinformatics/btv541,https://doi.org/10.1093/bioinformatics/btv540,https://doi.org/10.1093/bioinformatics/btv547,https://doi.org/10.1186/s12859-017-1779-5,https://doi.org/10.1093/nar/gkv1243
https://doi.org/10.1093/bioinformatics/btu599,1.000000,0.053514,0.025703,0.030648,0.076520,0.185043,0.045495,0.221082,0.042283,0.087531,...,0.075325,0.057006,0.061471,0.065891,0.086187,0.069803,0.073492,0.066914,0.004679,0.050045
https://doi.org/10.1093/database/baw006,0.053514,1.000000,0.011852,0.117884,0.043251,0.022455,0.026388,0.037234,0.041198,0.038123,...,0.042489,0.043266,0.156155,0.026372,0.072229,0.080570,0.030464,0.053564,0.079363,0.143249
https://doi.org/10.1093/bioinformatics/btu590,0.025703,0.011852,1.000000,0.026898,0.042381,0.021337,0.093871,0.046459,0.018969,0.037932,...,0.021177,0.012537,0.021716,0.046363,0.020060,0.021134,0.372726,0.022239,0.006491,0.031074
https://doi.org/10.1093/bioinformatics/btu593,0.030648,0.117884,0.026898,1.000000,0.029239,0.063387,0.080417,0.056976,0.036803,0.078307,...,0.037137,0.072769,0.068819,0.054828,0.106579,0.075205,0.059267,0.289757,0.213353,0.118639
https://doi.org/10.1093/bioinformatics/btu595,0.076520,0.043251,0.042381,0.029239,1.000000,0.127370,0.034770,0.052640,0.009911,0.042435,...,0.071257,0.017768,0.007665,0.068373,0.059575,0.022859,0.040674,0.031960,0.037842,0.048615


In [312]:
top_keyword_dataframe.head(50)

,word
doi,
https://doi.org/10.1093/bioinformatics/btu547,browser
https://doi.org/10.1093/bioinformatics/btu547,http
https://doi.org/10.1093/bioinformatics/btu547,plain
https://doi.org/10.1093/bioinformatics/btu547,web
https://doi.org/10.1093/bioinformatics/btu547,bam
https://doi.org/10.1093/bioinformatics/btu568,peak
https://doi.org/10.1093/bioinformatics/btu568,finder
https://doi.org/10.1093/bioinformatics/btu568,width
https://doi.org/10.1093/bioinformatics/btu568,enrich
